In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# %matplotlib notebook
# import ipympl
%matplotlib widget

import inspect

import models.ac_stark as stark
import models.rydberg_calcs as ryd
import plotters.plot_ac_stark as pac

from scipy.constants import c as C_c, h as C_h, k as C_k

In [14]:
from arc import Cesium as Cs
cs = Cs()

Pp = 1e-3
wp = 25e-6
qp = 1

Pc = 1
wc = 25e-6
qc = -1

Delta = np.array([0.5e9, 1e9, 2e9, 5e9, 10e9, 20e9])*2*np.pi #check units, seems like it wants rad/s
Delta = 1e9*2*np.pi

fg = 4
mfg = 4

ne = 7
le = 1
je = 3/2

nr = 41
lr = 0
jr = 1/2
mjr = 1/2

OmegaR, ACg, ACr, Psc = cs.twoPhotonRydbergExcitation(Pp, wp, qp, Pc, wc, qc, Delta, fg, mfg, ne, le, je, nr, lr, jr, mjr)

OmegaR, ACg, ACr, Psc = np.round(np.array([np.abs(OmegaR), ACg, ACr, Psc]) * 1e-6 / (2*np.pi), 3)
Omega_p = np.sqrt(ACg*4* Delta / (2*np.pi) *1e-6)
Omega_c = OmegaR / Omega_p * (Delta / (2*np.pi) *1e-6) * 2
print(OmegaR, -ACg, ACr, Psc)

print(Omega_p, Omega_c)

# plt.figure()
# plt.plot(Delta, np.sqrt(ACg*4* Delta / (2*np.pi) *1e-6))
# plt.figure()
# plt.plot(Delta, np.sqrt(ACr*4* Delta / (2*np.pi) *1e-6))

15.096 -2.95 141.804 0.0
108.62780491200216 277.9398886358618


-992233016.6285535 347640319.5826612 4818718361.714167 0.04013534823665278


In [22]:
transition41 = ryd.RydbergTransition(laserWaist=probe_waist, laserWaist2=couple_waist, n1=6, l1=0, j1=0.5,
                 mj1=0.5, n2=7, l2=1, j2=1.5, mj2=1.5, f2=5, q2=-1, n3=41, l3=0, j3=0.5, mj3=0.5, f3=4)

params = transition41.print_laser_frequencies(Pp=probe_power, Pc=couple_power)
probe_freq, couple_freq, two_photon_rabi, det, rabiFreq_1, rabiFreq_2 = params

Probe laser frequency (no AOM) 657932.3889647021 GHz
Probe laser frequency (with AOM) 657932.6089647021 GHz
Power Broadening $\sqrt(2)*\Omega = $ 440.3451306644454 MHz
Natural Linewidth 7.709815102267814 MHz

Couple laser frequency (no AOM) 281196.26994683413 GHz
Couple laser frequency (with AOM) 281196.4899468341 GHz
Power Broadening $\sqrt(2)*\Omega = $ 628.4152604495167 MHz
Natural Linewidth 0.029168477861902423 MHz

Optimal detuning 3.118835998346721 GHz 

Optimal probe frequency (with AOM) 657935.7278007006 GHz
Optimal couple frequency (with AOM) 281193.37111083575 GHz

Expected Rabi Frequency = 2*pi 22.181320220176822 MHz
Pi Pulse Duration 22.54148964249587 ns


In [23]:
def wavelength(frequency):
    #Frequency in GHz
    return C_c / frequency * 1e-9

print(wavelength(couple_freq))
print(couple_freq)
print(probe_freq)

1.0661442386334405e-06
281193.1511108358
657935.5078007005


In [43]:
#Calculate light shifts from lambdas with 
shift_calc = stark.ACStarkShift(laserWaist=couple_waist, n=7, l=1, j=1.5, mj=1.5, q=1)
shift = shift_calc.ac_stark_shift_shirley(wavelength(couple_freq), 2.8)
print(shift * 1e-6)
print(shift * C_h / C_k * 1e6)

ValueError: Target state [7, 1, 1.5, 1.5] not in states list